In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import cv2
from os import listdir

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def unique(list1): 
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set)) 
    return unique_list

import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
from statsmodels.stats import multitest

In [2]:
results_path = '/Users/margaretschroeder/Dropbox (MIT)/BoydenLab/ExR_Decrowding/Decrowding volume SNR/results/'

In [3]:
res_files = listdir(results_path)

In [4]:
res_files

['B3_syngap_shank_L1_result.xlsx',
 'A3_syngap_shank_L4_result.xlsx',
 'A5_rim_homer_L1_result.xlsx',
 'C7_shank_homer_L4_result.xlsx',
 'B4_homer_shank_L1_result.xlsx',
 'B6_bassoon_homer_L1_result.xlsx',
 'C7_shank_homer_L23_result.xlsx',
 'C5_rim_shank_L1_result.xlsx',
 'A4_homer_shank_L23_result.xlsx',
 'C1_cacha_shank_L23_result.xlsx',
 'B4_homer_shank_L4_result.xlsx',
 'B6_bassoon_homer_L4_result.xlsx',
 'A6_bassoon_homer_L23_result.xlsx',
 'B4_homer_shank_L23_result.xlsx',
 'C5_rim_shank_L4_result.xlsx',
 'A3_syngap_shank_L1_result.xlsx',
 'B3_syngap_shank_L4_result.xlsx',
 'B3_syngap_shank_L23_result.xlsx',
 'C7_shank_homer_L1_result.xlsx',
 'A5_rim_homer_L4_result.xlsx',
 'C4_homer_shank_L1_result.xlsx',
 'A2_psd95_homer_L1_result.xlsx',
 'A1_cacha_shank_L4_result.xlsx',
 'C6_bassoon_homer_L1_result.xlsx',
 'B7_shank_homer_L4_result.xlsx',
 'C3_syngap_shank_L23_result.xlsx',
 'B5_rim_homer_L23_result.xlsx',
 'B7_shank_homer_L1_result.xlsx',
 'A2_psd95_homer_L4_result.xlsx',
 '

In [5]:
results_path + res_files[0]

'/Users/margaretschroeder/Dropbox (MIT)/BoydenLab/ExR_Decrowding/Decrowding volume SNR/results/B3_syngap_shank_L1_result.xlsx'

## Calculate Post/Pre volume and SNR for plotting

In [6]:
data_all = pd.DataFrame(columns=['Sample_name','Sample','Layer','Protein','Vol_Post/Pre','SNR_Post/Pre'])
data_means = pd.DataFrame(columns=['Sample_name','Sample','Layer','Protein','Mean_Vol_Post/Pre','Mean_SNR_Post/Pre',
                                   'SEM_Vol_Post/Pre','SEM_SNR_Post/Pre'])
for file in res_files:
    dframe = pd.read_excel(results_path + file)
    dframe.dropna(inplace=True)
    length = len(dframe)
    
    datatemp = pd.DataFrame()
    datatemp['Sample_name'] = [file.split("_")[0]]*length
    datatemp['Sample'] = [file[0]]*length
    datatemp['Protein'] = [file.split("_")[1]]*length
    datatemp['Layer'] = [file.split("_")[3].split(".")[0]]*length
    datatemp['Protein-Layer'] = [file.split("_")[1] + "-" + file.split("_")[3].split(".")[0]]*length
    datatemp['Protein-Layer-Sample'] = [file.split("_")[1] + "-" + file.split("_")[3].split(".")[0] + "-" + file[0]]*length
    
    datatemp['Vol_Post/Pre'] = dframe['volume_post']/dframe['volume_pre']
    datatemp['SNR_Post/Pre'] = dframe['SNR_post']/dframe['SNR_pre']
    
    meantemp = pd.DataFrame()
    meantemp['Sample_name'] = [file.split("_")[0]]
    meantemp['Sample'] = [file[0]]
    meantemp['Protein'] = [file.split("_")[1]]
    meantemp['Layer'] = [file.split("_")[3].split(".")[0]]

    meantemp['Mean_Vol_Post/Pre'] = (dframe['volume_post']/dframe['volume_pre']).mean()
    meantemp['Mean_SNR_Post/Pre'] = (dframe['SNR_post']/dframe['SNR_pre']).mean()
    meantemp['SEM_Vol_Post/Pre'] = (dframe['volume_post']/dframe['volume_pre']).sem()
    meantemp['SEM_SNR_Post/Pre'] = (dframe['SNR_post']/dframe['SNR_pre']).sem()
    
    data_all = pd.concat([data_all,datatemp],axis=0,ignore_index=True)
    data_means = pd.concat([data_means,meantemp],axis=0,ignore_index=True)

In [7]:
data_all.sort_values(by="Sample_name",inplace=True)

In [8]:
data_all

,Sample_name,Sample,Layer,Protein,Vol_Post/Pre,SNR_Post/Pre,Protein-Layer,Protein-Layer-Sample
3677,A1,A,L23,cacha,25.200810,1.544792,cacha-L23,cacha-L23-A
1331,A1,A,L4,cacha,19.865169,2.173923,cacha-L4,cacha-L4-A
1332,A1,A,L4,cacha,13.171429,1.657710,cacha-L4,cacha-L4-A
1333,A1,A,L4,cacha,126.875000,1.747831,cacha-L4,cacha-L4-A
1334,A1,A,L4,cacha,81.875000,2.181853,cacha-L4,cacha-L4-A
...,...,...,...,...,...,...,...,...
378,C7,C,L23,shank,6.469256,2.082890,shank-L23,shank-L23-C
377,C7,C,L23,shank,8.916279,1.768680,shank-L23,shank-L23-C
376,C7,C,L23,shank,6.947368,1.619354,shank-L23,shank-L23-C
1115,C7,C,L1,shank,NaN,NaN,shank-L1,shank-L1-C


In [9]:
data_all.to_csv('decrowding_alldata_preoverpost.csv',index=False)

In [10]:
data_all.groupby("Protein-Layer-Sample").sem().to_clipboard()

In [11]:
data_all

,Sample_name,Sample,Layer,Protein,Vol_Post/Pre,SNR_Post/Pre,Protein-Layer,Protein-Layer-Sample
3677,A1,A,L23,cacha,25.200810,1.544792,cacha-L23,cacha-L23-A
1331,A1,A,L4,cacha,19.865169,2.173923,cacha-L4,cacha-L4-A
1332,A1,A,L4,cacha,13.171429,1.657710,cacha-L4,cacha-L4-A
1333,A1,A,L4,cacha,126.875000,1.747831,cacha-L4,cacha-L4-A
1334,A1,A,L4,cacha,81.875000,2.181853,cacha-L4,cacha-L4-A
...,...,...,...,...,...,...,...,...
378,C7,C,L23,shank,6.469256,2.082890,shank-L23,shank-L23-C
377,C7,C,L23,shank,8.916279,1.768680,shank-L23,shank-L23-C
376,C7,C,L23,shank,6.947368,1.619354,shank-L23,shank-L23-C
1115,C7,C,L1,shank,NaN,NaN,shank-L1,shank-L1-C


In [12]:
data_all.groupby("Protein-Layer-Sample").count()

,Sample_name,Sample,Layer,Protein,Vol_Post/Pre,SNR_Post/Pre,Protein-Layer
Protein-Layer-Sample,,,,,,,
bassoon-L1-A,58,58,58,58,56,56,58
bassoon-L1-B,57,57,57,57,55,55,57
bassoon-L1-C,65,65,65,65,63,63,65
bassoon-L23-A,58,58,58,58,56,56,58
bassoon-L23-B,58,58,58,58,56,56,58
...,...,...,...,...,...,...,...
syngap-L23-B,58,58,58,58,56,56,58
syngap-L23-C,65,65,65,65,63,63,65
syngap-L4-A,53,53,53,53,51,51,53


In [13]:
data_all

,Sample_name,Sample,Layer,Protein,Vol_Post/Pre,SNR_Post/Pre,Protein-Layer,Protein-Layer-Sample
3677,A1,A,L23,cacha,25.200810,1.544792,cacha-L23,cacha-L23-A
1331,A1,A,L4,cacha,19.865169,2.173923,cacha-L4,cacha-L4-A
1332,A1,A,L4,cacha,13.171429,1.657710,cacha-L4,cacha-L4-A
1333,A1,A,L4,cacha,126.875000,1.747831,cacha-L4,cacha-L4-A
1334,A1,A,L4,cacha,81.875000,2.181853,cacha-L4,cacha-L4-A
...,...,...,...,...,...,...,...,...
378,C7,C,L23,shank,6.469256,2.082890,shank-L23,shank-L23-C
377,C7,C,L23,shank,8.916279,1.768680,shank-L23,shank-L23-C
376,C7,C,L23,shank,6.947368,1.619354,shank-L23,shank-L23-C
1115,C7,C,L1,shank,NaN,NaN,shank-L1,shank-L1-C


## Redo statistics for 3-way ANOVA

In [14]:
data_all = pd.DataFrame(columns=['Sample_name','Sample','Layer','Protein','Vol_Pre','SNR_Pre','Vol_Post','SNR_Post'])

for file in res_files:
    #print(file)
    dframe = pd.read_excel(results_path + file)
    dframe.dropna(inplace=True)
    length = len(dframe)
    
    datatemp = pd.DataFrame()
    datatemp['Sample_name'] = [file.split("_")[0]]*length
    datatemp['Sample'] = [file[0]]*length
    datatemp['Protein'] = [file.split("_")[1]]*length
    datatemp['Layer'] = [file.split("_")[3].split(".")[0]]*length
    datatemp['Protein-Layer'] = [file.split("_")[1] + "-" + file.split("_")[3].split(".")[0]]*length
    
    datatemp['Vol_Pre'] = dframe['volume_pre']
    datatemp['SNR_Pre'] = dframe['SNR_pre']
    datatemp['Vol_Post'] = dframe['volume_post']
    datatemp['SNR_Post'] = dframe['SNR_post']
    
    data_all = pd.concat([data_all,datatemp],axis=0,ignore_index=True)

In [15]:
data_all

,Sample_name,Sample,Layer,Protein,Vol_Pre,SNR_Pre,Vol_Post,SNR_Post,Protein-Layer
0,B3,B,L1,syngap,475.0,10.911370,927.000000,17.236649,syngap-L1
1,B3,B,L1,syngap,1216.0,11.273987,2338.000000,21.867678,syngap-L1
2,B3,B,L1,syngap,1624.0,14.098815,2716.000000,25.974767,syngap-L1
3,B3,B,L1,syngap,1556.0,11.988611,1213.000000,14.760465,syngap-L1
4,B3,B,L1,syngap,2099.0,13.162335,2346.000000,22.617624,syngap-L1
...,...,...,...,...,...,...,...,...,...
3732,B6,B,L23,bassoon,2817.0,14.904318,1998.000000,17.703236,bassoon-L23
3733,B6,B,L23,bassoon,1595.0,11.272424,3972.000000,15.780428,bassoon-L23
3734,B6,B,L23,bassoon,NaN,NaN,NaN,NaN,bassoon-L23
3735,B6,B,L23,bassoon,1050.8,13.007976,2224.872727,18.426449,bassoon-L23


In [16]:
data_pre = data_all[['Vol_Pre','SNR_Pre','Sample','Protein','Layer']]
data_pre.rename(columns={"Vol_Pre": "Volume", "SNR_Pre": "SNR"},inplace=True)
data_post = data_all[['Vol_Post','SNR_Post','Sample','Protein','Layer']]
data_post.rename(columns={"Vol_Post": "Volume", "SNR_Post": "SNR"},inplace=True)
postlab = ['Post'] * len(data_post)
prelab = ['Pre'] * len(data_pre)
data_post['Pre_Post'] = postlab
data_pre['Pre_Post'] = prelab
data_new = pd.concat([data_pre,data_post],axis=0)

/Users/margaretschroeder/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/margaretschroeder/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/margaretschroeder/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [17]:
data_new

,Volume,SNR,Sample,Protein,Layer,Pre_Post
0,475.000000,10.911370,B,syngap,L1,Pre
1,1216.000000,11.273987,B,syngap,L1,Pre
2,1624.000000,14.098815,B,syngap,L1,Pre
3,1556.000000,11.988611,B,syngap,L1,Pre
4,2099.000000,13.162335,B,syngap,L1,Pre
...,...,...,...,...,...,...
3732,1998.000000,17.703236,B,bassoon,L23,Post
3733,3972.000000,15.780428,B,bassoon,L23,Post
3734,NaN,NaN,B,bassoon,L23,Post
3735,2224.872727,18.426449,B,bassoon,L23,Post


In [18]:
data_vol = data_new[['Volume','Protein','Layer','Pre_Post','Sample']]
data_snr = data_new[['SNR','Protein','Layer','Pre_Post','Sample']]

In [19]:
data_vol

,Volume,Protein,Layer,Pre_Post,Sample
0,475.000000,syngap,L1,Pre,B
1,1216.000000,syngap,L1,Pre,B
2,1624.000000,syngap,L1,Pre,B
3,1556.000000,syngap,L1,Pre,B
4,2099.000000,syngap,L1,Pre,B
...,...,...,...,...,...
3732,1998.000000,bassoon,L23,Post,B
3733,3972.000000,bassoon,L23,Post,B
3734,NaN,bassoon,L23,Post,B
3735,2224.872727,bassoon,L23,Post,B


In [20]:
## Volume
lm_full = ols('Volume ~ C(Protein) + C(Layer) + C(Pre_Post) + C(Protein)*C(Pre_Post) + C(Layer)*C(Protein)',data=data_vol).fit()
table_full = sm.stats.anova_lm(lm_full, typ=3)
print(table_full)
table_full.to_clipboard()

                              sum_sq      df            F         PR(>F)
Intercept               1.304201e+09     1.0  1073.448035  1.411791e-219
C(Protein)              8.251541e+08     6.0   113.193181  5.732194e-137
C(Layer)                2.093032e+06     2.0     0.861355   4.226325e-01
C(Pre_Post)             5.508082e+08     1.0   453.353404   1.312898e-97
C(Protein):C(Pre_Post)  7.984413e+08     6.0   109.528762  1.261601e-132
C(Layer):C(Protein)     2.246174e+08    12.0    15.406311   1.209022e-32
Residual                8.742884e+09  7196.0          NaN            NaN


In [21]:
##SNR
lm_full = ols('SNR ~ C(Protein) + C(Layer) + C(Pre_Post) + + C(Protein)*C(Pre_Post) + C(Layer)*C(Protein)',data=data_snr).fit()
table_full = sm.stats.anova_lm(lm_full, typ=3)
print(table_full)
table_full.to_clipboard()

                               sum_sq      df            F         PR(>F)
Intercept                84727.131760     1.0  5890.145970   0.000000e+00
C(Protein)               20088.098968     6.0   232.750778  1.217757e-272
C(Layer)                   424.350995     2.0    14.750230   4.047172e-07
C(Pre_Post)              11041.591235     1.0   767.600446  1.267454e-160
C(Protein):C(Pre_Post)   22260.585164     6.0   257.922292  8.563761e-300
C(Layer):C(Protein)       1092.360641    12.0     6.328319   2.836256e-11
Residual                103511.261562  7196.0          NaN            NaN


## Now run multiple 2-way ANOVAs for each protein, and correct these

In [ ]:
prot_list = ['bassoon','cacha','homer','psd95','rim','shank','syngap']

In [32]:
#volume
pvals = []
for prot in prot_list:
    prot_subset = data_vol[data_vol['Protein']==prot]
    #print(prot_subset)
    lm = ols('Volume ~ C(Pre_Post) + C(Layer)',data=prot_subset).fit()
    tbl = sm.stats.anova_lm(lm, typ=3)
    print(tbl)
    p = tbl.loc['C(Pre_Post)']['PR(>F)']
    pvals.append(p)
    print(prot)
    print(p)

                   sum_sq      df            F         PR(>F)
Intercept    1.304201e+09     1.0  1381.415956  8.789011e-194
C(Pre_Post)  5.508082e+08     1.0   583.418671  6.689043e-103
C(Layer)     2.093032e+06     2.0     1.108475   3.304472e-01
Residual     9.931980e+08  1052.0          NaN            NaN
bassoon
6.689043253785397e-103
                   sum_sq     df           F         PR(>F)
Intercept    1.230211e+08    1.0  867.255997  1.085724e-137
C(Pre_Post)  1.002173e+08    1.0  706.496793  3.913269e-118
C(Layer)     5.558294e+06    2.0   19.592020   4.509031e-09
Residual     1.415673e+08  998.0         NaN            NaN
cacha
3.9132687577883205e-118
                   sum_sq      df           F         PR(>F)
Intercept    6.344816e+08     1.0  580.846630  1.136919e-101
C(Pre_Post)  3.171345e+07     1.0   29.032600   8.864420e-08
C(Layer)     8.073347e+07     2.0   36.954392   3.259015e-16
Residual     1.098893e+09  1006.0         NaN            NaN
homer
8.864420236182163e

In [33]:
pvalarr = np.array(pvals)
reject,pvalscorr,_,_ = multitest.multipletests(pvalarr, alpha=0.05, method='hs', is_sorted=False, returnsorted=False)
corrected_multiways = pd.DataFrame([pvalarr,pvalscorr,reject])
corrected_multiways.T.to_clipboard()
corrected_multiways.T

,0,1,2
0,6.68904e-103,0,True
1,3.91327e-118,0,True
2,8.86442e-08,8.86442e-08,True
3,1.04021e-98,0,True
4,1.27778e-112,0,True
5,6.52928e-33,0,True
6,3.63031e-71,0,True


In [31]:
#volume
pvals = []
for prot in prot_list:
    prot_subset = data_snr[data_snr['Protein']==prot]
    #print(prot_subset)
    lm = ols('SNR ~ C(Pre_Post) + C(Layer)',data=prot_subset).fit()
    tbl = sm.stats.anova_lm(lm, typ=3)
    print(tbl)
    p = tbl.loc['C(Pre_Post)']['PR(>F)']
    pvals.append(p)
    print(prot)
    print(p)

pvalarr = np.array(pvals)
reject,pvalscorr,_,_ = multitest.multipletests(pvalarr, alpha=0.05, method='hs', is_sorted=False, returnsorted=False)
corrected_multiways = pd.DataFrame([pvalarr,pvalscorr,reject])
corrected_multiways.T.to_clipboard()
corrected_multiways.T

                   sum_sq      df             F         PR(>F)
Intercept    84727.131760     1.0  11714.927180   0.000000e+00
C(Pre_Post)  11041.591235     1.0   1526.682594  4.890573e-207
C(Layer)       424.350995     2.0     29.336771   3.997778e-13
Residual      7608.493099  1052.0           NaN            NaN
bassoon
4.8905730085368936e-207
                   sum_sq     df            F         PR(>F)
Intercept    54576.346748    1.0  2917.910510  1.616559e-298
C(Pre_Post)   4387.312143    1.0   234.566529   1.029141e-47
C(Layer)       493.875157    2.0    13.202455   2.191725e-06
Residual     18666.506002  998.0          NaN            NaN
cacha
1.029141435979728e-47
                   sum_sq      df            F         PR(>F)
Intercept    94245.267573     1.0  7500.426165   0.000000e+00
C(Pre_Post)   7564.497882     1.0   602.013866  1.432775e-104
C(Layer)       123.281005     2.0     4.905605   7.583065e-03
Residual     12640.713620  1006.0          NaN            NaN
homer
1.43

,0,1,2
0,4.89057e-207,0,True
1,1.02914e-47,0,True
2,1.43278e-104,0,True
3,1.93537e-107,0,True
4,1.58566e-158,0,True
5,4.39708e-71,0,True
6,2.2051e-245,0,True
